In [1]:
import torch
normal_path = "/workspace/MILForVideos/extractors/resnet/datasets/UCSD_Anomaly_Dataset_v1p2_ped1_test_normal.pt"
video = torch.load(normal_path)[0]
video.features

tensor([[[-2.9645, -2.3521, -0.4318,  ..., -3.8537,  0.7928, -0.1892]],

        [[-3.3547, -2.5740, -0.2007,  ..., -4.0114,  0.4159, -0.5973]],

        [[-3.5241, -2.4136, -0.4487,  ..., -4.1432,  0.4215, -0.2618]],

        ...,

        [[-4.1561, -3.4499, -1.0856,  ..., -4.5663, -0.4501, -0.1749]],

        [[-3.9828, -3.5695, -1.1362,  ..., -4.3238, -0.1868, -0.2681]],

        [[-3.9444, -3.3779, -0.9227,  ..., -4.3789, -0.5177, -0.3598]]])

In [2]:
video.features.size()

torch.Size([200, 1, 1000])

In [3]:
video.path_list[:1]

['/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/001.tif']

In [4]:
from PIL import Image
from extractor import MyNet, img2tensor

idx = 0

net = MyNet()
net = net.eval().cuda()
with torch.no_grad():
    image = img2tensor(video.path_list[idx]).cuda()
    feature = net(image)[0, 0].cpu()
    display(video.features[idx, 0, ~(video.features[idx, 0] == feature)].size())

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


torch.Size([927])

In [24]:
video.features[idx, 0, -10:].float()

tensor([ 2.1255, -0.9766, -3.3938, -3.5765, -1.9704, -1.4774, -1.2138, -3.8537,
         0.7928, -0.1892])

In [31]:
feature[-10:].float()

tensor([ 2.1255, -0.9766, -3.3938, -3.5765, -1.9704, -1.4774, -1.2138, -3.8537,
         0.7928, -0.1892])

In [38]:
display(video.features[idx, 0, -10:][0].numpy())
display(feature[-10:][0].numpy())
display(type(video.features[idx, 0, -10:][0]))
display(type(feature[-10:][0].numpy()))

video.features[idx, 0, -10:][0] == feature[-10:][0]

array(2.1254601, dtype=float32)

array(2.1254613, dtype=float32)

torch.Tensor

numpy.ndarray

tensor(False)

In [7]:
import sys
import torch
import torch.nn as nn
import argparse
import pandas as pd
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from milforvideo.video import Extractor


pathlist = "/datasets/UCSD_Anomaly_Dataset_v1p2/ped1_test_normal.txt"

# Get the image path and label from a input file
with open(pathlist) as f:
    grp_path_and_label = [
        row.split(" ")
        for row in f.read().split("\n")
        if row
    ]
    df = pd.DataFrame({
        "grp": [int(grp) for grp, _, _ in grp_path_and_label],
        "path": [path for _, path, _ in grp_path_and_label],
        "label": [int(label) for _, _, label in grp_path_and_label],
    })

net = MyNet()
outputs = []
grp, df_grp = list(df.groupby("grp"))[0]
extractor = Extractor(
    df_grp["path"].tolist(), 
    df_grp["label"].tolist(), 
    net, img2tensor,
    aggregate=lambda label: [label], 
    cuda=True)
new_video = extractor.extract()
new_video.features

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


tensor([[[-2.9645, -2.3521, -0.4318,  ..., -3.8537,  0.7928, -0.1892]],

        [[-3.3547, -2.5740, -0.2007,  ..., -4.0114,  0.4159, -0.5973]],

        [[-3.5241, -2.4136, -0.4487,  ..., -4.1432,  0.4215, -0.2618]],

        ...,

        [[-4.1561, -3.4499, -1.0856,  ..., -4.5663, -0.4501, -0.1749]],

        [[-3.9828, -3.5695, -1.1362,  ..., -4.3238, -0.1868, -0.2681]],

        [[-3.9444, -3.3779, -0.9227,  ..., -4.3789, -0.5177, -0.3598]]])

In [8]:
display(video.features[0, 0, ~(video.features[0, 0] == new_video.features[0, 0])].size())

torch.Size([0])

In [9]:
video.features[0, 0, -10:]

tensor([ 2.1255, -0.9766, -3.3938, -3.5765, -1.9704, -1.4774, -1.2138, -3.8537,
         0.7928, -0.1892])

In [10]:
new_video.features[0, 0, -10:]

tensor([ 2.1255, -0.9766, -3.3938, -3.5765, -1.9704, -1.4774, -1.2138, -3.8537,
         0.7928, -0.1892])